In [1]:
PATH = 'C:\\Users\\krenovut\\Desktop\\rubeacon\\client-academia\\'
# PATH = 'C:\\Users\\vadim\\Desktop\\RuBeacon\\Clients-Datasets\\client-academia\\'

In [86]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.cluster import KMeans
from datetime import datetime
import json

In [3]:
def read_multiples(file_name, number):
    res = pd.DataFrame()
    for i in range(1, number+1):
        path = PATH + file_name + '-' + str(i) + '.csv'
        f = pd.read_csv(path)
        res = pd.concat([res,f])
    return res

In [4]:
items = pd.read_csv(PATH + "academia-items.csv")
cats = pd.read_csv(PATH + "academia-cats.csv")
order_items = read_multiples('academia-order-items', 9)
orders = read_multiples('academia-orders', 8)
orders['date_created'] = pd.to_datetime(orders['date_created'],
                              format='%Y-%m-%d %H:%M:%S %Z')

In [5]:
items.head()

,item_id,cat_id,title,price,picture
0,04a5cb4b-da08-aa0a-0153-a9d681ae413b,04a5cb4b-da08-aa0a-0153-a9d681ae3f15,Раф цитрусовый холодный,150,http://empatika-resto.appspot.com/img/aHR0cHM6...
1,04a5cb4b-da08-aa0a-0153-a9d681ae419f,2057d6a9-7387-4a14-b42f-7b922f52540b,Бейгл с лососем,170,http://empatika-resto.appspot.com/img/aHR0cHM6...
2,04a5cb4b-da08-aa0a-0153-a9d681ae41f5,04a5cb4b-da08-aa0a-0153-a9d681ae3f30,Эфиопия Арича Мытая 100гр,180,NaN
3,04a5cb4b-da08-aa0a-0153-a9d681ae4202,04a5cb4b-da08-aa0a-0153-a9d681ae3f3b,Латте большой,140,http://empatika-resto.appspot.com/img/aHR0cHM6...
4,04a5cb4b-da08-aa0a-0153-a9d681ae4207,a77eacd0-6036-4791-a22c-fe96da4a27e0,Чай Зеленый Чун Ми,120,http://empatika-resto.appspot.com/img/aHR0cHM6...


In [6]:
cats.head()

,cat_id,parent_cat_id,title,picture
0,04a5cb4b-da08-aa0a-0153-a9d681ae3f03,5720147234914304,Кофе в розницу,http://empatika-resto.appspot.com/img/aHR0cHM6...
1,1de039ca-388b-4d4e-abd8-21e63a6ed0cf,04a5cb4b-da08-aa0a-0153-a9d681ae3f43,Соки,http://empatika-resto.appspot.com/img/aHR0cHM6...
2,5743573328723968,5720147234914304,Зимнее предложение,http://lh3.googleusercontent.com/j2RZet7-yl4Pd...
3,04a5cb4b-da08-aa0a-0153-a9d681ae3f36,04a5cb4b-da08-aa0a-0153-a9d681ae3f43,Шоколадные,http://empatika-resto.appspot.com/img/aHR0cHM6...
4,04a5cb4b-da08-aa0a-0153-a9d681ae3f3b,04a5cb4b-da08-aa0a-0153-a9d681ae3f38,Классический,http://empatika-resto.appspot.com/img/aHR0cHM6...


In [7]:
order_items.head()

,index,client_id,order_id,item_id,item_cnt
0,1,692412,117796,04a5cb4b-da08-aa0a-0153-a9d681ae4246,1
1,2,1886815,117797,a0498a99-3932-4830-9e51-dda6cb506e9c,1
2,3,1886815,117797,af3110d3-e8b7-411e-b4c3-f841911ff6ee,1
3,4,1675612,117798,04a5cb4b-da08-aa0a-0153-a9d681ae421f,1
4,5,2116411,117799,b8967389-1777-405d-8cb0-d56c25599b72,1


In [8]:
orders.head()

,index,order_id,client_id,total_sum,date_created
0,1,10,10871,200.0,2015-11-18 14:57:05
1,2,11,10406,110.0,2015-11-19 09:31:51
2,3,117796,692412,110.0,2018-10-05 12:53:57
3,4,117797,1886815,280.0,2018-10-05 13:00:13
4,5,117798,1675612,130.0,2018-10-05 13:01:47


## Дата / число дней с первого заказа

#### SQL
SELECT client_id, date_created FROM orders  
ORDER BY date_created

In [9]:
orders.head()

,index,order_id,client_id,total_sum,date_created
0,1,10,10871,200.0,2015-11-18 14:57:05
1,2,11,10406,110.0,2015-11-19 09:31:51
2,3,117796,692412,110.0,2018-10-05 12:53:57
3,4,117797,1886815,280.0,2018-10-05 13:00:13
4,5,117798,1675612,130.0,2018-10-05 13:01:47


In [10]:
orders[27730:27733]

,index,order_id,client_id,total_sum,date_created
12730,27731,5,10406,425.0,2015-11-18 09:56:12
12731,27732,5000017740300288,2190876,930.0,2018-07-19 02:04:58
12732,27733,5000023696211968,502416,235.0,2018-03-26 04:47:22


In [11]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114863 entries, 0 to 9862
Data columns (total 5 columns):
index           114863 non-null int64
order_id        114863 non-null int64
client_id       114863 non-null int64
total_sum       114863 non-null float64
date_created    114863 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 5.3 MB


In [12]:
sort_orders = orders.sort_values(by=['date_created'])

In [13]:
sort_orders.head()

,index,order_id,client_id,total_sum,date_created
4494,4495,2,10871,225.0,2015-11-16 10:53:22
4502,4503,3,10871,350.0,2015-11-17 08:39:43
4509,4510,4,10871,60.0,2015-11-17 12:07:46
12730,27731,5,10406,425.0,2015-11-18 09:56:12
4701,79702,6,10406,130.0,2015-11-18 14:12:20


In [84]:
sort_orders['client_id'][:5]

4494     10871
4502     10871
4509     10871
12730    10406
4701     10406
Name: client_id, dtype: int64

In [15]:
client_info = {}

for index, row in sort_orders.iterrows():
    tmp = []
    if row['client_id'] in client_info:
        client_info[row['client_id']].append(row['date_created'])
    else:
        tmp.append(row['date_created'])
        client_info[row['client_id']] = tmp

In [93]:
client_info;

In [92]:
{x:client_info[x] for x in [10871, 11234]}

{10871: [Timestamp('2015-11-16 10:53:22'),
  Timestamp('2015-11-17 08:39:43'),
  Timestamp('2015-11-17 12:07:46'),
  Timestamp('2015-11-18 14:57:05'),
  Timestamp('2015-12-06 10:09:13'),
  Timestamp('2015-12-18 09:21:18')],
 11234: [Timestamp('2015-11-19 15:37:42'),
  Timestamp('2015-11-19 15:40:20'),
  Timestamp('2015-11-19 16:23:30')]}

In [17]:
right_now = datetime.now()
time_from_first_order = {}
for client in client_info:
    time_from_first_order[client] = client_info[client][0] - right_now

In [94]:
time_from_first_order;

In [95]:
{x:time_from_first_order[x] for x in [10871, 11234]}

{10871: -1285, 11234: -1281}

## Дата / число дней с последнего заказа

In [19]:
right_now = datetime.now()
time_from_last_order = {}
for client in client_info:
    time_from_last_order[client] = client_info[client][-1] - right_now

In [97]:
time_from_last_order;

In [96]:
{x:time_from_last_order[x] for x in [10871, 11234]}

{10871: -1253, 11234: -1281}

## Среднее число дней между заказами

In [21]:
def get_mean_time(client_info):
    mean_time_between_orders = {}
    for client in client_info:
        timestamps = client_info[client]
        if len(timestamps) == 1:
            mean_time_between_orders[client] = 0
            continue
        cnt = 0
        last_date = timestamps[0] 
        time_between = 0
        for time in timestamps[1:]:
            diff = time - last_date
            time_between += diff.days
            cnt += 1
        mean_time_between_orders[client] = time_between/cnt
    return mean_time_between_orders

In [22]:
mean_time_between_orders = get_mean_time(client_info)

In [98]:
mean_time_between_orders;

In [99]:
{x:mean_time_between_orders[x] for x in [10871, 11234]}

{10871: 10.6, 11234: 0.0}

## средняя стоимость заказа

#### SQL
SELECT client_id, SUM(total_sum), COUNT(*), SUM(total_sum)/ (COUNT(*)*1.0) FROM orders  
GROUP BY client_id;

In [24]:
client_and_sum = orders[['client_id', 'total_sum']]
client_and_sum.head()

,client_id,total_sum
0,10871,200.0
1,10406,110.0
2,692412,110.0
3,1886815,280.0
4,1675612,130.0


In [25]:
client_and_sum = client_and_sum.set_index('client_id')
client_and_sum.head()

,total_sum
client_id,
10871,200.0
10406,110.0
692412,110.0
1886815,280.0
1675612,130.0


In [100]:
client_and_sum.groupby('client_id').size();

In [27]:
grouped_client_sum = client_and_sum.groupby('client_id').sum()
grouped_client_count = client_and_sum.groupby('client_id').count()

In [28]:
grouped_client = pd.merge(left=grouped_client_sum, right=grouped_client_count, how='outer', left_index=True, right_index=True)

In [29]:
grouped_client.columns = ['total_sum', 'total_count']
grouped_client.head()

,total_sum,total_count
client_id,,
10406,10075.0,31
10871,1145.0,6
11229,4015.0,25
11234,1135.0,3
13768,60.0,1


In [30]:
# grouped_client['mean_order'] = (grouped_client.total_sum)/(grouped_ client.total_count)

In [31]:
grouped_client.head()

,total_sum,total_count
client_id,,
10406,10075.0,31
10871,1145.0,6
11229,4015.0,25
11234,1135.0,3
13768,60.0,1


## Число заказов по дням недели 

In [32]:
orders.head()

,index,order_id,client_id,total_sum,date_created
0,1,10,10871,200.0,2015-11-18 14:57:05
1,2,11,10406,110.0,2015-11-19 09:31:51
2,3,117796,692412,110.0,2018-10-05 12:53:57
3,4,117797,1886815,280.0,2018-10-05 13:00:13
4,5,117798,1675612,130.0,2018-10-05 13:01:47


In [33]:
ord_weekdays = orders[['index','order_id','client_id','date_created']]
ord_weekdays.head()

,index,order_id,client_id,date_created
0,1,10,10871,2015-11-18 14:57:05
1,2,11,10406,2015-11-19 09:31:51
2,3,117796,692412,2018-10-05 12:53:57
3,4,117797,1886815,2018-10-05 13:00:13
4,5,117798,1675612,2018-10-05 13:01:47


In [34]:
ord_weekdays['weekday'] = ord_weekdays['date_created'].dt.dayofweek

In [35]:
ord_weekdays['days_name'] = ord_weekdays[['date_created']].apply(lambda x: datetime.strftime(x['date_created'], '%A'), axis=1)

In [36]:
ord_weekdays.head()

,index,order_id,client_id,date_created,weekday,days_name
0,1,10,10871,2015-11-18 14:57:05,2,Wednesday
1,2,11,10406,2015-11-19 09:31:51,3,Thursday
2,3,117796,692412,2018-10-05 12:53:57,4,Friday
3,4,117797,1886815,2018-10-05 13:00:13,4,Friday
4,5,117798,1675612,2018-10-05 13:01:47,4,Friday


In [37]:
ord_weekdays[ord_weekdays['client_id'] == 10871]

,index,order_id,client_id,date_created,weekday,days_name
0,1,10,10871,2015-11-18 14:57:05,2,Wednesday
4494,4495,2,10871,2015-11-16 10:53:22,0,Monday
4499,4500,26,10871,2015-12-06 10:09:13,6,Sunday
4502,4503,3,10871,2015-11-17 08:39:43,1,Tuesday
4509,4510,4,10871,2015-11-17 12:07:46,1,Tuesday
2610,32611,51,10871,2015-12-18 09:21:18,4,Friday


In [38]:
test = ord_weekdays[['order_id', 'client_id', 'date_created', 'weekday', 'days_name']]

In [39]:
group = test.groupby(['client_id', 'days_name']).size()

In [40]:
tmp = group.head(15)
tmp

client_id  days_name
10406      Friday       3
           Monday       5
           Saturday     1
           Sunday       3
           Thursday     7
           Tuesday      7
           Wednesday    5
10871      Friday       1
           Monday       1
           Sunday       1
           Tuesday      2
           Wednesday    1
11229      Friday       3
           Monday       5
           Sunday       4
dtype: int64

#### SQL
select client_id,  DATEFORMAT(date_created, 'EEEE'), COUNT(DATEFORMAT(date_created, 'EEEE')) from orders  
GROUP BY client_id, DATEFORMAT(date_created, 'EEEE')


In [41]:
for ind, val in tmp.items():
    print(ind[0],ind[1], val)

10406 Friday 3
10406 Monday 5
10406 Saturday 1
10406 Sunday 3
10406 Thursday 7
10406 Tuesday 7
10406 Wednesday 5
10871 Friday 1
10871 Monday 1
10871 Sunday 1
10871 Tuesday 2
10871 Wednesday 1
11229 Friday 3
11229 Monday 5
11229 Sunday 4


In [42]:
from dateutil import rrule

In [43]:
def weeks_between(start_date, end_date):
    weeks = rrule.rrule(rrule.WEEKLY, dtstart=start_date, until=end_date)
    return weeks.count()

orders_time = client_info[10406]
number_of_weeks_between = 1
if len(orders_time) == 1:
    number_of_weeks_between = 1
else:
    number_of_weeks_between = weeks_between(orders_time[0], orders_time[-1])
# orders_time
number_of_weeks_between

28

In [101]:
buy_days = {}
for ind, val in group.items():
    client_id = ind[0]
    
    orders_time = client_info[client_id]
    number_of_weeks_between = 1
    if len(orders_time) == 1:
        number_of_weeks_between = 1
    else:
        number_of_weeks_between = weeks_between(orders_time[0], orders_time[-1])
    
    weekday = ind[1]
    if client_id in buy_days:
        val = val/number_of_weeks_between
        buy_days[client_id][weekday] = val
    else:
        val = val/number_of_weeks_between
        buy_days[client_id] = {weekday : val}

buy_days;
{x:buy_days[x] for x in [10871, 11234]}

{10871: {'Friday': 0.20000000000000001,
  'Monday': 0.20000000000000001,
  'Sunday': 0.20000000000000001,
  'Tuesday': 0.40000000000000002,
  'Wednesday': 0.20000000000000001},
 11234: {'Thursday': 3.0}}

## Заказы по времени суток
Разделил на утро (с 6 до 11), день (с 11 до 18) и вечер/ночь (с 18 до 6)

In [45]:
orders.head()

,index,order_id,client_id,total_sum,date_created
0,1,10,10871,200.0,2015-11-18 14:57:05
1,2,11,10406,110.0,2015-11-19 09:31:51
2,3,117796,692412,110.0,2018-10-05 12:53:57
3,4,117797,1886815,280.0,2018-10-05 13:00:13
4,5,117798,1675612,130.0,2018-10-05 13:01:47


In [46]:
ord_times = orders[['index','order_id','client_id','date_created']]
ord_times.head()

,index,order_id,client_id,date_created
0,1,10,10871,2015-11-18 14:57:05
1,2,11,10406,2015-11-19 09:31:51
2,3,117796,692412,2018-10-05 12:53:57
3,4,117797,1886815,2018-10-05 13:00:13
4,5,117798,1675612,2018-10-05 13:01:47


In [47]:
ord_times['hour'] = ord_times['date_created'].dt.hour

#### SQL
SELECT client_id, HOUR(date_created) FROM orders  

In [48]:
ord_times.head()

,index,order_id,client_id,date_created,hour
0,1,10,10871,2015-11-18 14:57:05,14
1,2,11,10406,2015-11-19 09:31:51,9
2,3,117796,692412,2018-10-05 12:53:57,12
3,4,117797,1886815,2018-10-05 13:00:13,13
4,5,117798,1675612,2018-10-05 13:01:47,13


In [49]:
conditions = [
    (ord_times['hour'] >= 6) & (ord_times['hour'] < 11),
     (ord_times['hour'] >= 11) & (ord_times['hour'] <18),
    (ord_times['hour'] >= 18) & (ord_times['hour'] < 6)]
choices = ['morning', 'afternoon', 'evening/night']
ord_times['times_of_day'] = np.select(conditions, choices, default="evening/night")

In [50]:
ord_times.head()

,index,order_id,client_id,date_created,hour,times_of_day
0,1,10,10871,2015-11-18 14:57:05,14,afternoon
1,2,11,10406,2015-11-19 09:31:51,9,morning
2,3,117796,692412,2018-10-05 12:53:57,12,afternoon
3,4,117797,1886815,2018-10-05 13:00:13,13,afternoon
4,5,117798,1675612,2018-10-05 13:01:47,13,afternoon


In [51]:
times = ord_times.groupby(['client_id', 'times_of_day']).size()
times.head()

client_id  times_of_day 
10406      afternoon        14
           evening/night     2
           morning          15
10871      afternoon         2
           morning           4
dtype: int64

In [102]:
buy_times = {}
for ind, val in times.items():
    client_id = ind[0]
    weekday = ind[1]
    if client_id in buy_times:
        buy_times[client_id][weekday] = val
    else:
        buy_times[client_id] = {weekday : val}
buy_times;
{x:buy_times[x] for x in [10871, 11234]}

{10871: {'afternoon': 2, 'morning': 4}, 11234: {'afternoon': 3}}

### Кол-во товаров купленные клиентом вообще

#### SQL
SELECT client_id, SUM(item_cnt) FROM order_items  
GROUP BY client_id

In [105]:
tmp = order_items[['client_id', 'order_id', 'item_cnt']]
tmp = tmp.groupby(['client_id', 'item_cnt']).size()
client_item_cnt = {}
for client,cnt in tmp.items():
    if client[0] in client_item_cnt:
        client_item_cnt[client[0]] += client[1]*cnt
    else:
        client_item_cnt[client[0]] = client[1]*cnt

client_item_cnt;
{x:client_item_cnt[x] for x in [120340, 213811]}

{120340: 3, 213811: 15}

### Товары

In [54]:
a = order_items[['client_id', 'item_id', 'item_cnt']]
a[a['client_id'] == 267251]

,client_id,item_id,item_cnt
10044,267251,04a5cb4b-da08-aa0a-0153-a9d681ae423f,2
10045,267251,58e38e11-2959-4a86-a382-53ac6c923830,2
11004,267251,04a5cb4b-da08-aa0a-0153-a9d681ae41fd,1
11005,267251,04a5cb4b-da08-aa0a-0153-a9d681ae423f,1
48,267251,04a5cb4b-da08-aa0a-0153-a9d681ae4227,2
49,267251,58e38e11-2959-4a86-a382-53ac6c923830,2
12402,267251,04a5cb4b-da08-aa0a-0153-a9d681ae41fd,1
10885,267251,04a5cb4b-da08-aa0a-0153-a9d681ae41fd,1


In [55]:
item_cat = items[['item_id', 'cat_id']]
cat_par = cats[['cat_id', 'parent_cat_id']]
merge_items = item_cat.join(cat_par.set_index('cat_id'), on='cat_id')
merge_items['parent_cat_id'].value_counts()

04a5cb4b-da08-aa0a-0153-a9d681ae3f38    36
6574506961272832                        32
5720147234914304                        32
d7f9378f-8a90-4e58-8663-f63ce0115acd    27
04a5cb4b-da08-aa0a-0153-a9d681ae3f43    23
5755685136498688                        23
04a5cb4b-da08-aa0a-0153-a9d681ae3f03    22
4819823842295808                        21
04a5cb4b-da08-aa0a-0153-a9d681ae3f46    17
5644406560391168                        14
Name: parent_cat_id, dtype: int64

#### SQL
SELECT oi.client_id, c.parent_cat_id, SUM(oi.item_cnt) FROM order_items AS oi, items AS i, cats AS c  
WHERE oi.item_id = i.item_id AND i.cat_id = c.cat_id  
GROUP BY oi.client_id, c.parent_cat_id  

In [108]:
tmp = order_items[['client_id', 'item_id', 'item_cnt']]
tmp['item_id'] = merge_items['parent_cat_id']
tmp = tmp.groupby(['client_id', 'item_id', 'item_cnt']).size()
# tmp

client_item = {}
for client,cnt in tmp.items():
    if client[0] in client_item: 
        client_item[client[0]][client[1]] = client[2]*cnt
    else:
        client_item[client[0]] = {client[1] : client[2]*cnt}
        
client_item;
{x:client_item[x] for x in [223648, 267251]}

C:\Users\krenovut\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


{223648: {'04a5cb4b-da08-aa0a-0153-a9d681ae3f03': 1,
  '5720147234914304': 1,
  'd7f9378f-8a90-4e58-8663-f63ce0115acd': 1},
 267251: {'04a5cb4b-da08-aa0a-0153-a9d681ae3f43': 2,
  'd7f9378f-8a90-4e58-8663-f63ce0115acd': 2}}

## Немного рефакторинга, приведем все в функции

In [57]:
def prepare_orders(orders):
    ''' на вход получаем orders
        на выходе словарь {client_id : [время заказов]}
    '''
    sort_orders = orders.sort_values(by=['date_created'])
    client_info = {}

    for index, row in sort_orders.iterrows():
        tmp = []
        if row['client_id'] in client_info:
            client_info[row['client_id']].append(row['date_created'])
        else:
            tmp.append(row['date_created'])
            client_info[row['client_id']] = tmp
    return client_info
client_orders_time = prepare_orders(orders)

In [58]:
# Дата / число дней с первого заказа
def get_days_from_first_order(client_orders_time):
    right_now = datetime.now()
    time_from_first_order = {}
    for client in client_orders_time:
        time_from_first_order[client] = (client_orders_time[client][0] - right_now).days
    return time_from_first_order
time_from_first_order = get_days_from_first_order(client_orders_time)

In [114]:
time_from_first_order;

In [115]:
# Дата / число дней с последнего заказа
def get_days_from_last_order(client_orders_time):
    right_now = datetime.now()
    time_from_last_order = {}
    for client in client_orders_time:
        time_from_last_order[client] = (client_orders_time[client][-1] - right_now).days
    return time_from_last_order
time_from_last_order = get_days_from_last_order(client_orders_time)

In [116]:
time_from_last_order;

In [117]:
# Среднее число дней между заказами
def get_mean_time(client_orders_time):
    mean_time_between_orders = {}
    for client in client_orders_time:
        timestamps = client_orders_time[client]
        if len(timestamps) == 1:
            mean_time_between_orders[client] = 0
            continue
        cnt = 0
        last_date = timestamps[0] 
        time_between = 0
        for time in timestamps[1:]:
            diff = time - last_date
            time_between += diff.days
            cnt += 1
        mean_time_between_orders[client] = time_between/cnt
    return mean_time_between_orders
mean_time_between_orders = get_mean_time(client_orders_time)

In [118]:
mean_time_between_orders;

In [64]:
# средняя стоимость заказа
def get_all_and_mean_price(orders):
    client_and_sum = orders[['client_id', 'total_sum']]
    client_and_sum = client_and_sum.set_index('client_id')
    grouped_client_sum = client_and_sum.groupby('client_id').sum()
    grouped_client_count = client_and_sum.groupby('client_id').count()
    grouped_client = pd.merge(left=grouped_client_sum, right=grouped_client_count, how='outer', left_index=True, right_index=True)
    grouped_client.columns = ['total_sum', 'total_count']
    grouped_client['mean_order'] = (grouped_client.total_sum)/(grouped_client.total_count)
    client_total_order = dict(zip(grouped_client.index, grouped_client['total_sum'].values))
    client_mean_order = dict(zip(grouped_client.index, grouped_client['mean_order'].values))
    return client_total_order, client_mean_order
# grouped_client = get_mean_price(orders)
# grouped_client.head(10)
client_total_order, client_mean_order = get_all_and_mean_price(orders)


In [109]:
client_total_order, client_mean_order;

In [66]:
# Число заказов по дням недели
def weeks_between(start_date, end_date):
    weeks = rrule.rrule(rrule.WEEKLY, dtstart=start_date, until=end_date)
    return weeks.count()

def get_number_of_orders_per_day_of_week(orders, client_orders_time):
    ord_weekdays = orders[['index','order_id','client_id','date_created']]
    ord_weekdays['weekday'] = ord_weekdays['date_created'].dt.dayofweek
    ord_weekdays['days_name'] = ord_weekdays[['date_created']].apply(lambda x: datetime.strftime(x['date_created'], '%A'), axis=1)
    group = ord_weekdays[['order_id', 'client_id', 'date_created', 'weekday', 'days_name']].groupby(['client_id', 'days_name']).size()
    buy_days = {}
    for ind, val in group.items():
        client_id = ind[0]
        orders_time = client_orders_time[client_id]
        number_of_weeks_between = 1
        if len(orders_time) == 1:
            number_of_weeks_between = 1
        else:
            number_of_weeks_between = weeks_between(orders_time[0], orders_time[-1])

        weekday = ind[1]
        if client_id in buy_days:
            val = val/number_of_weeks_between
            buy_days[client_id][weekday] = val
        else:
            val = val/number_of_weeks_between
            buy_days[client_id] = {weekday : val}
    return buy_days
buy_days = get_number_of_orders_per_day_of_week(orders, client_orders_time)

In [110]:
buy_days;

In [68]:
# Заказы по времени суток
# Разделил на утро (с 6 до 11), день (с 11 до 18) и вечер/ночь (с 18 до 6)
def get_number_of_orders_by_time_of_day(orders):
    ord_times = orders[['index','order_id','client_id','date_created']]
    ord_times['hour'] = ord_times['date_created'].dt.hour
    conditions = [
        (ord_times['hour'] >= 6) & (ord_times['hour'] < 11),
        (ord_times['hour'] >= 11) & (ord_times['hour'] <18),
        (ord_times['hour'] >= 18) & (ord_times['hour'] < 6)
    ]
    choices = ['morning', 'afternoon', 'evening/night']
    ord_times['times_of_day'] = np.select(conditions, choices, default="evening/night")
    ord_times = ord_times.groupby(['client_id', 'times_of_day']).size()
    buy_times = {}
    for ind, val in ord_times.items():
        client_id = ind[0]
        weekday = ind[1]
        if client_id in buy_times:
            buy_times[client_id][weekday] = val
        else:
            buy_times[client_id] = {weekday : val}
    return buy_times
buy_times = get_number_of_orders_by_time_of_day(orders)

In [111]:
buy_times;

In [112]:
def get_number_of_items_purchased_by_the_client(order_items):
    tmp = order_items[['client_id', 'order_id', 'item_cnt']]
    tmp = tmp.groupby(['client_id', 'item_cnt']).size()
    client_item_cnt = {}
    for client,cnt in tmp.items():
        if client[0] in client_item_cnt:
            client_item_cnt[client[0]] += client[1]*cnt
        else:
            client_item_cnt[client[0]] = client[1]*cnt
    return client_item_cnt

client_item_cnt;

In [71]:
def prepare_items(items, cats, order_items):
    
    item_cat = items[['item_id', 'cat_id']]
    cat_par = cats[['cat_id', 'parent_cat_id']]
    merge_items = item_cat.join(cat_par.set_index('cat_id'), on='cat_id')

    tmp = order_items[['client_id', 'item_id', 'item_cnt']]
    tmp['item_id'] = merge_items['parent_cat_id']
    tmp = tmp.groupby(['client_id', 'item_id', 'item_cnt']).size()
    # tmp

    client_item = {}
    for client,cnt in tmp.items():
        if client[0] in client_item: 
            client_item[client[0]][client[1]] = client[2]*cnt
        else:
            client_item[client[0]] = {client[1] : client[2]*cnt}
    return client_item
        

In [113]:
client_item = prepare_items(items, cats, order_items)
client_item;

C:\Users\krenovut\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Создатим таблицу по клиентам со всеми данными

In [73]:
from collections import defaultdict
dd = defaultdict(list)
for d in time_from_first_order,time_from_last_order, mean_time_between_orders, client_total_order, client_mean_order, client_item_cnt:
    for key, value in iter(d.items()):
        dd[key].append(value)

In [74]:
dd[120340]

[-879, -224, 207.78947368421052, 2854.5, 142.72499999999999, 3]

In [75]:
df = pd.DataFrame.from_dict(dd, orient='index')
df.columns = ['from first', 'from last', 'mean time', 'total sum', 'mean sum', 'cnt_items']

In [76]:
df.head()

,from first,from last,mean time,total sum,mean sum,cnt_items
10871,-1285,-1253,10.600000,1145.0,190.833333,NaN
10406,-1283,-1089,62.766667,10075.0,325.000000,NaN
11234,-1281,-1281,0.000000,1135.0,378.333333,NaN
13768,-1264,-1264,0.000000,60.0,60.000000,NaN
14361,-1263,-1259,1.888889,1490.0,149.000000,NaN


In [77]:
df_days = pd.DataFrame.from_dict( buy_days, orient='index')
df_days.fillna(0, inplace=True)
df_days.head()

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
10406,0.107143,0.178571,0.035714,0.107143,0.250000,0.250000,0.178571
10871,0.200000,0.200000,0.000000,0.200000,0.000000,0.400000,0.200000
11229,0.058824,0.098039,0.000000,0.078431,0.058824,0.156863,0.039216
11234,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000
13768,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [78]:
df_time = pd.DataFrame.from_dict( buy_times, orient='index')
df_time.fillna(0, inplace=True)
df_time.head()

,afternoon,evening/night,morning
10406,14.0,2.0,15.0
10871,2.0,0.0,4.0
11229,11.0,8.0,6.0
11234,3.0,0.0,0.0
13768,0.0,0.0,1.0


In [79]:
df_items = pd.DataFrame.from_dict(client_item, orient='index')
df_items.fillna(0, inplace=True)
df_items.head()

,04a5cb4b-da08-aa0a-0153-a9d681ae3f03,5720147234914304,d7f9378f-8a90-4e58-8663-f63ce0115acd,04a5cb4b-da08-aa0a-0153-a9d681ae3f43,04a5cb4b-da08-aa0a-0153-a9d681ae3f38,4819823842295808,5755685136498688,6574506961272832,5644406560391168,04a5cb4b-da08-aa0a-0153-a9d681ae3f46
223648,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
267251,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
267298,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
267347,0.0,1.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
267759,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [80]:
tmp = pd.concat([df_days, df_time, df_items], axis=1)
tmp.head()

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,afternoon,evening/night,morning,04a5cb4b-da08-aa0a-0153-a9d681ae3f03,5720147234914304,d7f9378f-8a90-4e58-8663-f63ce0115acd,04a5cb4b-da08-aa0a-0153-a9d681ae3f43,04a5cb4b-da08-aa0a-0153-a9d681ae3f38,4819823842295808,5755685136498688,6574506961272832,5644406560391168,04a5cb4b-da08-aa0a-0153-a9d681ae3f46
10406,0.107143,0.178571,0.035714,0.107143,0.250000,0.250000,0.178571,14.0,2.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10871,0.200000,0.200000,0.000000,0.200000,0.000000,0.400000,0.200000,2.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11229,0.058824,0.098039,0.000000,0.078431,0.058824,0.156863,0.039216,11.0,8.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11234,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13768,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
df_for_clust = pd.concat([df, tmp], axis=1)
df_for_clust.fillna(0, inplace=True)
df_for_clust.head()

,from first,from last,mean time,total sum,mean sum,cnt_items,Friday,Monday,Saturday,Sunday,...,04a5cb4b-da08-aa0a-0153-a9d681ae3f03,5720147234914304,d7f9378f-8a90-4e58-8663-f63ce0115acd,04a5cb4b-da08-aa0a-0153-a9d681ae3f43,04a5cb4b-da08-aa0a-0153-a9d681ae3f38,4819823842295808,5755685136498688,6574506961272832,5644406560391168,04a5cb4b-da08-aa0a-0153-a9d681ae3f46
10406,-1283,-1089,62.766667,10075.0,325.000000,0.0,0.107143,0.178571,0.035714,0.107143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10871,-1285,-1253,10.600000,1145.0,190.833333,0.0,0.200000,0.200000,0.000000,0.200000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11229,-1099,-742,168.333333,4015.0,160.600000,0.0,0.058824,0.098039,0.000000,0.078431,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11234,-1281,-1281,0.000000,1135.0,378.333333,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13768,-1264,-1264,0.000000,60.0,60.000000,0.0,0.000000,1.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
